<a href="https://colab.research.google.com/github/ATLAS097/Data-Engineering-CA2/blob/Swam/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning
---
### Date: 21 July 2025
### Class: DAAA/FT/2B/22
### Group-5
### Group Member: Kaung Myat San, Swam Htet Aung, Ethan Ng Wen Xian
---

## Loading Modules

In [1]:
!pip install python-docx

import pandas as pd
from docx import Document
import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.9 MB/s eta 0:00:00


## Loading Data
---

In [2]:
# read appointment.xlsx from a URL (must use the raw file link and specify engine)
appointment = pd.read_excel(
	"https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/appointment.xlsx",
	engine="openpyxl"
)

In [3]:
!wget https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/facility.docx -O facility.docx
document = Document("facility.docx")

table = document.tables[0]

data = []
for row in table.rows:
    data.append([cell.text for cell in row.cells])

# Convert to pandas DataFrame
facility = pd.DataFrame(data)

# Set the first row as header and remove it from data
facility.columns = facility.iloc[0]
facility = facility[1:].reset_index(drop=True)
# rename BuldingID to BuildingID
facility.rename(columns={'BuldingID': 'BuildingID'}, inplace=True)

display(facility.head())

--2025-07-30 16:43:58--  https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/facility.docx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34082 (33K) [application/octet-stream]
Saving to: ‘facility.docx’

facility.docx       100%[===================>]  33.28K  --.-KB/s    in 0.008s  

2025-07-30 16:43:58 (4.15 MB/s) - ‘facility.docx’ saved [34082/34082]



,Department,BuildingID,BuildingName,RoomID,Purpose
0,Emergency,B0001,Emergency 1,R0101,Consultation Room
1,Emergency,B0001,Emergency 1,R0102,Consultation Room
2,Emergency,B0001,Emergency 1,R0103,Emergency Room
3,Emergency,B0001,Emergency 1,R0104,Pharmacy
4,Emergency,B0001,Emergency 1,R0201,Laboratory


In [4]:
patient = pd.read_excel("https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/patient.xlsx", engine="openpyxl")
patient.head()

,PatientID,Name,Sex,Contact,email,Address,DOB
0,P0001,Kelsey Moore,female,39472495,kelsey.moore@gmail.com,"34892 Christopher Crossing, South Brooke, NH 1...",1997-05-05
1,P0002,William Perry,male,15351117,william.perry@hotmail.com,"94501 Bell Islands Suite 485, New Jessica, NJ ...",2007-06-06
2,P0003,Nicholas Jones,male,56648257,nicholas.jones@yahoo.com,"8841 Lewis Parkways Apt. 040, South Kristen, I...",1975-12-04
3,P0004,Theresa Smith,female,45807263,theresa.smith@outlook.com,"271 Jeffrey Freeway, Willisbury, ME 39487",1997-09-14
4,P0005,Tara Kennedy,female,84234563,tara.kennedy@protonmail.com,"64128 Rick Coves Apt. 284, Lake James, NJ 51964",2011-02-22


In [5]:
staff = pd.read_csv("https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/staff.csv")
staff.head()

,DoctorID,Name,Contact,Email,DepartmentID,Department
0,E0001,Grace Chen,78901234,grace.chen@abchospital.com,D0005,Pediatrics
1,E0002,Henry Adams,75678901,henry.adams@abchospital.com,D0002,Intensive Care Unit
2,E0003,Maya Patel,44567890,maya.patel@abchospital.com,D0008,Surgery
3,E0004,Sean Underwood,84345678,sean.underwood@abchospital.com,D0001,Emergency
4,E0005,Amy Bryant,72123456,amy.bryant@abchospital.com,D0009,Neurology


# Facility Data Analysis and Cleaning
---

In [6]:
facility.head()

,Department,BuildingID,BuildingName,RoomID,Purpose
0,Emergency,B0001,Emergency 1,R0101,Consultation Room
1,Emergency,B0001,Emergency 1,R0102,Consultation Room
2,Emergency,B0001,Emergency 1,R0103,Emergency Room
3,Emergency,B0001,Emergency 1,R0104,Pharmacy
4,Emergency,B0001,Emergency 1,R0201,Laboratory


In [7]:
# Check for duplicate rows based on 'BuildingID', 'BuildingName', and 'RoomID'
duplicate_facility_subset = facility.duplicated(subset=['BuildingID', 'BuildingName', 'RoomID'], keep=False)

# Filter the DataFrame to show only the duplicate rows
duplicate_facility_rows = facility[duplicate_facility_subset]

print("Rows with duplicate 'BuildingID', 'BuildingName', and 'RoomID' combinations:")
if not duplicate_facility_rows.empty:
    display(duplicate_facility_rows)
else:
    print("No duplicate combinations of 'BuildingID', 'BuildingName', and 'RoomID' found.")

# To check for duplicates in other columns for these rows, you would visually inspect the output above.
# If you want to programmatically check for duplicates in a specific other column for these rows,
# you could do something like this:
# print("\nChecking for duplicates in 'Purpose' for the identified duplicate rows:")
# display(duplicate_facility_rows[duplicate_facility_rows.duplicated(subset=['BuildingID', 'BuildingName', 'RoomID', 'Purpose'], keep=False)])

Rows with duplicate 'BuildingID', 'BuildingName', and 'RoomID' combinations:
No duplicate combinations of 'BuildingID', 'BuildingName', and 'RoomID' found.


In [8]:

duplicate_rows = facility.duplicated()
num_duplicates = duplicate_rows.sum()

print(f"\nNumber of duplicate rows found: {num_duplicates}")

# If duplicates exist, display them
if num_duplicates > 0:
    print("\nHere are the duplicate rows:")
    # The 'keep=False' argument shows all occurrences of duplicates
    print(facility[facility.duplicated(keep=False)])
else:
    print("✅ There are no duplicate rows in the dataset.")


Number of duplicate rows found: 0
✅ There are no duplicate rows in the dataset.


In [9]:
print("\n--- Unique Values in Each Column ---")

for column in facility.columns:
    print(f"\n✅ Column: '{column}'")

    # Get and print the unique values
    unique_values = facility[column].unique()
    print(unique_values)


--- Unique Values in Each Column ---

✅ Column: 'Department'
['Emergency' 'Cardiology' 'Pediatrics' 'Orthopedics'
 'Obstetrics and Gynecology' 'Surgery' 'Neurology' 'Oncology' 'Radiology'
 'Intensive Care Unit' '']

✅ Column: 'BuildingID'
['B0001 ' 'B0002 ' 'B0003 ' 'B0004 ' '']

✅ Column: 'BuildingName'
['Emergency 1' 'Central Tower' 'ICU Tower' 'Emergency 2' '']

✅ Column: 'RoomID'
[' R0101' ' R0102' ' R0103' ' R0104' ' R0201' ' R0202' ' R0203' ' R0204'
 ' R0301' ' R0302' ' R0303' ' R0304' ' R0401' ' R0402' ' R0403' ' R0404'
 '']

✅ Column: 'Purpose'
['Consultation Room' 'Emergency Room' 'Pharmacy' 'Laboratory' 'X-ray Room'
 'Operating Room' 'Treatment Room' 'Critical Care Room'
 'Isolation ICU Room' 'MRI Room' '']


In [10]:
facility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Department    49 non-null     object
 1   BuildingID    49 non-null     object
 2   BuildingName  49 non-null     object
 3   RoomID        49 non-null     object
 4   Purpose       49 non-null     object
dtypes: object(5)
memory usage: 2.0+ KB


There are no null values.

In [11]:
print(facility.tail(1))


0  Department BuildingID BuildingName RoomID Purpose
48                                                  


I remove the last row since it's blank.

In [12]:
facility = facility.iloc[:-1]


# Appointment Data Analysis and Cleaning
---

In [13]:
appointment

,PatientID,DoctorID,BuildingID,RoomID,ScheduledStart,ScheduledEnd,Arrival,ActualStart,ActualEnd,Weight(kg),Temperature(celcius),Comment
0,P0191,E0066,B0001,R0402,2024-01-05 14:00:00,2024-01-05 15:00:00,2024-01-05 14:35:00,2024-01-05 14:35:00,2024-01-05 15:11:00,49.500000,37.200000,Orientation comfortably
1,P0776,E0065,B0004,R0104,2024-02-27 14:00:00,2024-02-27 15:00:00,2024-02-27 14:59:00,2024-02-27 14:59:00,2024-02-27 16:03:00,55.500000,37.100000,Imaging has improved slightly
2,P0157,E0051,B0001,R0301,2022-12-28 09:00:00,2022-12-28 10:00:00,NaT,NaT,NaN,NaN,NaN,NaN
3,P0593,E0044,B0004,R0101,2023-01-04 15:00:00,2023-01-04 16:00:00,2023-01-04 15:48:00,2023-01-04 15:48:00,2023-01-04 16:22:00,50.500000,36.800000,Rash reported today
4,P0434,E0074,B0002,R0203,2020-04-14 09:00:00,2020-04-14 10:00:00,2020-04-14 09:39:00,2020-04-14 09:39:00,2020-04-14 10:07:00,49.500000,38.100000,Continue observed on forearm
...,...,...,...,...,...,...,...,...,...,...,...,...
99996,P0025,E0050,B0002,R0102,2021-01-27 10:00:00,2021-01-27 11:00:00,2021-01-27 09:33:00,2021-01-27 10:00:00,2021-01-27 10:34:00,52.500000,37.100000,Imaging remains unchanged
99997,P0127,E0059,B0002,R0402,2023-12-04 14:00:00,2023-12-04 15:00:00,2023-12-04 14:09:00,2023-12-04 14:09:00,2023-12-04 15:23:00,51.500000,37.100000,Heart rate in one week
99998,P0565,E0066,B0004,R0201,2020-01-03 09:00:00,2020-01-03 10:00:00,2020-01-03 08:03:00,2020-01-03 09:00:00,2020-01-03 09:42:00,50.500000,38.900000,Resting steady and unlabored
99999,P0600,E0011,B0001,R0304,2022-07-26 09:00:00,2022-07-26 10:00:00,2022-07-26 09:20:00,2022-07-26 09:20:00,2022-07-26 09:41:00,51.500000,37.200000,Wound site in one week


In [14]:
print("Number of rows in the appointment DataFrame:", len(appointment))

Number of rows in the appointment DataFrame: 100001


### Checking for nulls


In [15]:
appointment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100001 entries, 0 to 100000
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   PatientID             100000 non-null  object        
 1   DoctorID              100000 non-null  object        
 2   BuildingID            100000 non-null  object        
 3   RoomID                100000 non-null  object        
 4   ScheduledStart        100000 non-null  datetime64[ns]
 5   ScheduledEnd          100000 non-null  datetime64[ns]
 6   Arrival               97952 non-null   datetime64[ns]
 7   ActualStart           97952 non-null   datetime64[ns]
 8   ActualEnd             97953 non-null   object        
 9   Weight(kg)            97953 non-null   float64       
 10  Temperature(celcius)  97953 non-null   float64       
 11  Comment               97952 non-null   object        
dtypes: datetime64[ns](4), float64(2), object(6)
memory usage: 

From looking at the non null count, it seems there is a row where it is null for Arrival, ActualStart, Comment but not for ActualEnd, Weight and Temperature. I will examine it further.


In [16]:
# Rows where not all arrival, actualstart, actual end, weight, temperature and comment is all null or not null
cols = ['Arrival', 'ActualStart', 'ActualEnd', 'Weight(kg)', 'Temperature(celcius)', 'Comment']

not_null_rows = appointment[appointment[cols].notnull().all(axis=1)]
print("Count of rows where all specified col is not null:", len(not_null_rows))
selected_rows_null_rows = appointment[appointment[cols].isnull().all(axis=1)]
print("Count of rows where all specified col is null:", len(selected_rows_null_rows))
# Rows where NOT all are null AND NOT all are not null
mixed_rows = appointment[
    ~(appointment[cols].isnull().all(axis=1)) &
    ~(appointment[cols].notnull().all(axis=1))
]
print("Count of rows where not all specified col is null but all other col is null(the ones not specified):", len(mixed_rows))
appointment[appointment.index.isin(mixed_rows.index)]

Count of rows where all specified col is not null: 97952
Count of rows where all specified col is null: 2048
Count of rows where not all specified col is null but all other col is null(the ones not specified): 1


,PatientID,DoctorID,BuildingID,RoomID,ScheduledStart,ScheduledEnd,Arrival,ActualStart,ActualEnd,Weight(kg),Temperature(celcius),Comment
100000,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,average,51.497356,37.140657,NaN


From looking above, we can see that there is a row where data is missing, therefore we will remove it.


In [17]:
# remove rows where index is in mixed_rows
appointment = appointment[~appointment.index.isin(mixed_rows.index)]
appointment.info()


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   PatientID             100000 non-null  object        
 1   DoctorID              100000 non-null  object        
 2   BuildingID            100000 non-null  object        
 3   RoomID                100000 non-null  object        
 4   ScheduledStart        100000 non-null  datetime64[ns]
 5   ScheduledEnd          100000 non-null  datetime64[ns]
 6   Arrival               97952 non-null   datetime64[ns]
 7   ActualStart           97952 non-null   datetime64[ns]
 8   ActualEnd             97952 non-null   object        
 9   Weight(kg)            97952 non-null   float64       
 10  Temperature(celcius)  97952 non-null   float64       
 11  Comment               97952 non-null   object        
dtypes: datetime64[ns](4), float64(2), object(6)
memory usage: 9.9+ M

#### PatientID, DoctorID, BuildingID, RoomID
---


In [18]:
# Check data types before cleaning
print("=== DATA TYPES BEFORE CLEANING ===")
print("Appointment dataset:")
print(f"  DoctorID: {appointment['DoctorID'].dtype}")
print(f"  PatientID: {appointment['PatientID'].dtype}")
print(f"  BuildingID: {appointment['BuildingID'].dtype}")
print(f"  RoomID: {appointment['RoomID'].dtype}")

print("\nStaff dataset:")
print(f"  DoctorID: {staff['DoctorID'].dtype}")

print("\nPatient dataset:")
print(f"  PatientID: {patient['PatientID'].dtype}")

print("\nFacility dataset:")
print(f"  BuildingID: {facility['BuildingID'].dtype}")  # Note: facility uses 'BuildingID' (typo in original)
print(f"  RoomID: {facility['RoomID'].dtype}")

print("\n" + "="*50)

# Clean whitespaces
print("=== CLEANING WHITESPACES ===")

# Appointment dataset
appointment['DoctorID'] = appointment['DoctorID'].str.strip()
appointment['PatientID'] = appointment['PatientID'].str.strip()
appointment['BuildingID'] = appointment['BuildingID'].str.strip()
appointment['RoomID'] = appointment['RoomID'].str.strip()

# Staff dataset
staff['DoctorID'] = staff['DoctorID'].str.strip()

# Patient dataset
patient['PatientID'] = patient['PatientID'].str.strip()

# Facility dataset
facility['BuildingID'] = facility['BuildingID'].str.strip()
facility['RoomID'] = facility['RoomID'].str.strip()

print("Whitespace cleaning completed!")

print("\n" + "="*50)

# Check data types after cleaning
print("=== DATA TYPES AFTER CLEANING ===")
print("Appointment dataset:")
print(f"  DoctorID: {appointment['DoctorID'].dtype}")
print(f"  PatientID: {appointment['PatientID'].dtype}")
print(f"  BuildingID: {appointment['BuildingID'].dtype}")
print(f"  RoomID: {appointment['RoomID'].dtype}")

print("\nStaff dataset:")
print(f"  DoctorID: {staff['DoctorID'].dtype}")

print("\nPatient dataset:")
print(f"  PatientID: {patient['PatientID'].dtype}")

print("\nFacility dataset:")
print(f"  BuildingID: {facility['BuildingID'].dtype}")
print(f"  RoomID: {facility['RoomID'].dtype}")

=== DATA TYPES BEFORE CLEANING ===
Appointment dataset:
  DoctorID: object
  PatientID: object
  BuildingID: object
  RoomID: object

Staff dataset:
  DoctorID: object

Patient dataset:
  PatientID: object

Facility dataset:
  BuildingID: object
  RoomID: object

=== CLEANING WHITESPACES ===
Whitespace cleaning completed!

=== DATA TYPES AFTER CLEANING ===
Appointment dataset:
  DoctorID: object
  PatientID: object
  BuildingID: object
  RoomID: object

Staff dataset:
  DoctorID: object

Patient dataset:
  PatientID: object

Facility dataset:
  BuildingID: object
  RoomID: object


In [19]:
# --- Forward Checks ---
print("=== Forward Checks (Appointment -> Supporting Datasets) ===")

# Check if PatientID in appointment exists in patient
patient_id_check = appointment['PatientID'].isin(patient['PatientID']).all()
print(f"All PatientIDs in appointment are in patient dataset: {patient_id_check}")

# Check if DoctorID in appointment exists in staff
doctor_id_check = appointment['DoctorID'].isin(staff['DoctorID']).all()
print(f"All DoctorIDs in appointment are in staff dataset: {doctor_id_check}")

# Check if BuildingID in appointment exists in facility
building_id_check = appointment['BuildingID'].isin(facility['BuildingID']).all()
print(f"All BuildingIDs in appointment are in facility dataset: {building_id_check}")

# Check if RoomID in appointment exists in facility
room_id_check = appointment['RoomID'].isin(facility['RoomID']).all()
print(f"All RoomIDs in appointment are in facility dataset: {room_id_check}")

# Check if the combined BuildingID and RoomID from appointment exist in facility
appointment['BuildingRoom'] = appointment['BuildingID'] + appointment['RoomID']
facility['BuildingRoom'] = facility['BuildingID'] + facility['RoomID']
building_room_check = appointment['BuildingRoom'].isin(facility['BuildingRoom']).all()
print(f"All BuildingID and RoomID combinations in appointment are in facility: {building_room_check}")

# --- Reverse Checks ---
print("\n=== Reverse Checks (Supporting Datasets -> Appointment) ===")

# Check if all PatientIDs in patient are used in appointment
patient_id_reverse_check = patient['PatientID'].isin(appointment['PatientID']).all()
print(f"All PatientIDs in patient dataset are used in appointment: {patient_id_reverse_check}")

# Check if all DoctorIDs in staff are used in appointment
doctor_id_reverse_check = staff['DoctorID'].isin(appointment['DoctorID']).all()
print(f"All DoctorIDs in staff dataset are used in appointment: {doctor_id_reverse_check}")

# Check if all BuildingIDs in facility are used in appointment
building_id_reverse_check = facility['BuildingID'].isin(appointment['BuildingID']).all()
print(f"All BuildingIDs in facility are used in appointment: {building_id_reverse_check}")

# Check if all RoomIDs in facility are used in appointment
room_id_reverse_check = facility['RoomID'].isin(appointment['RoomID']).all()
print(f"All RoomIDs in facility are used in appointment: {room_id_reverse_check}")

# Check if all BuildingRoom combinations in facility are used in appointment
building_room_reverse_check = facility['BuildingRoom'].isin(appointment['BuildingRoom']).all()
print(f"All BuildingID and RoomID combinations in facility are used in appointment: {building_room_reverse_check}")

# Drop the temporary combined columns
appointment.drop(columns=['BuildingRoom'], inplace=True)
facility.drop(columns=['BuildingRoom'], inplace=True)


=== Forward Checks (Appointment -> Supporting Datasets) ===
All PatientIDs in appointment are in patient dataset: True
All DoctorIDs in appointment are in staff dataset: True
All BuildingIDs in appointment are in facility dataset: False
All RoomIDs in appointment are in facility dataset: True
All BuildingID and RoomID combinations in appointment are in facility: False

=== Reverse Checks (Supporting Datasets -> Appointment) ===
All PatientIDs in patient dataset are used in appointment: True
All DoctorIDs in staff dataset are used in appointment: False
All BuildingIDs in facility are used in appointment: False
All RoomIDs in facility are used in appointment: False
All BuildingID and RoomID combinations in facility are used in appointment: False


Based on our referential integrity checks, we found mismatches between datasets — for example, some DoctorID, BuildingID, and RoomID values are not consistently present across all datasets.
To retain as much data as possible, we might need to combine records from both sides rather than strictly filtering based on one primary dataset.

While the appointment dataset has the largest number of rows (100,000), our goal is to maximize usable records by merging intelligently and keeping entries that have valid references in either direction wherever appropriate.


* **DoctorID** → Use `staff` dataset
* **PatientID** → Can use both `appointment` and `patient` datasets
* **BuildingID** → Further examination needed
* **RoomID** → Use `facility` dataset


In [20]:
# Filter rows where BuildingID in appointment is NOT in facility
building_id_only_in_appointment = appointment[~appointment['BuildingID'].isin(facility['BuildingID'])]

# Display full rows
print("\nBuildingIDs found only in appointment, not in facility:")
print(building_id_only_in_appointment)




BuildingIDs found only in appointment, not in facility:
      PatientID DoctorID BuildingID RoomID      ScheduledStart  \
44299     P0029    E0015      BB002  R0203 2020-09-13 09:00:00   

             ScheduledEnd             Arrival         ActualStart  \
44299 2020-09-13 10:00:00 2020-09-13 09:16:00 2020-09-13 09:16:00   

                 ActualEnd  Weight(kg)  Temperature(celcius)  \
44299  2020-09-13 09:38:00        50.5                  38.6   

                           Comment  
44299  Mobility shows no discharge  


This is most likely be error, BB002 should B0002.

In [21]:
# Filter rows where BuildingID in facility is NOT in appointment
building_id_only_in_facility = facility[~facility['BuildingID'].isin(appointment['BuildingID'])]

# Display result
print("\nBuildingIDs found only in facility, not in appointment:")
print(building_id_only_in_facility[['BuildingID', 'RoomID']])



BuildingIDs found only in facility, not in appointment:
0  BuildingID RoomID
32      B0003  R0101
33      B0003  R0102
34      B0003  R0103
35      B0003  R0104
36      B0003  R0201
37      B0003  R0202
38      B0003  R0203
39      B0003  R0204


From this observation, we will note facility dataset as a dataset that includes all buildingId, assuming BB002 is error.

In [22]:
# Create combined key in both DataFrames
appointment['BuildingRoom'] = appointment['BuildingID'] + appointment['RoomID']
facility['BuildingRoom'] = facility['BuildingID'] + facility['RoomID']

# Forward check: appointment → facility
missing_in_facility = appointment[~appointment['BuildingRoom'].isin(facility['BuildingRoom'])]
print("\nRows in appointment where BuildingID + RoomID not found in facility:")
print(missing_in_facility)

# Reverse check: facility → appointment
missing_in_appointment = facility[~facility['BuildingRoom'].isin(appointment['BuildingRoom'])]
print("\nRows in facility where BuildingID + RoomID not found in appointment:")
print(missing_in_appointment)

# Optional: Clean up temporary column
appointment.drop(columns=['BuildingRoom'], inplace=True)
facility.drop(columns=['BuildingRoom'], inplace=True)



Rows in appointment where BuildingID + RoomID not found in facility:
      PatientID DoctorID BuildingID RoomID      ScheduledStart  \
44299     P0029    E0015      BB002  R0203 2020-09-13 09:00:00   

             ScheduledEnd             Arrival         ActualStart  \
44299 2020-09-13 10:00:00 2020-09-13 09:16:00 2020-09-13 09:16:00   

                 ActualEnd  Weight(kg)  Temperature(celcius)  \
44299  2020-09-13 09:38:00        50.5                  38.6   

                           Comment BuildingRoom  
44299  Mobility shows no discharge   BB002R0203  

Rows in facility where BuildingID + RoomID not found in appointment:
0            Department BuildingID   BuildingName RoomID             Purpose  \
3             Emergency      B0001    Emergency 1  R0104            Pharmacy   
4             Emergency      B0001    Emergency 1  R0201          Laboratory   
5             Emergency      B0001    Emergency 1  R0202          Laboratory   
6             Emergency      B0001    E

From the row where the combination of BuildingID and RoomID wasn't found
- it seems it is beacuse of spelling error where `BBOO2` was inserted instead of `B0002`.
- Therefore, we will change it.

In [23]:
# Correct the BuildingID for the specific row
appointment.loc[44299, 'BuildingID'] = 'B0002'

# Verify the change
display(appointment.loc[44299])

,44299
PatientID,P0029
DoctorID,E0015
BuildingID,B0002
RoomID,R0203
ScheduledStart,2020-09-13 09:00:00
ScheduledEnd,2020-09-13 10:00:00
Arrival,2020-09-13 09:16:00
ActualStart,2020-09-13 09:16:00
ActualEnd,2020-09-13 09:38:00
Weight(kg),50.5


#### Scheduled_Start, Scheduled_End, Arrival, Actual_Start, Actual_End
---

In [24]:
# Convert 'ActualEnd' to datetime, coercing errors
appointment['ActualEnd'] = pd.to_datetime(appointment['ActualEnd'], errors='coerce')

# Display info to confirm changes
# appointment.info()

# Check for dates outside a reasonable range
# Determine a reasonable date range based on ScheduledStart and ScheduledEnd
min_scheduled_date = appointment['ScheduledStart'].min()
max_scheduled_date = appointment['ScheduledEnd'].max()

print(f"Earliest Scheduled Start Date: {min_scheduled_date}")
print(f"Latest Scheduled End Date: {max_scheduled_date}")

# Define a buffer period (e.g., 1 year before the earliest scheduled start and 1 year after the latest scheduled end)
date_buffer = pd.DateOffset(years=1)
earliest_reasonable_date = min_scheduled_date - date_buffer
latest_reasonable_date = max_scheduled_date + date_buffer

date_columns = ['ScheduledStart', 'ScheduledEnd', 'Arrival', 'ActualStart', 'ActualEnd']

for col in date_columns:
    # Check for dates before the earliest reasonable date
    invalid_earlier_dates = appointment[appointment[col] < earliest_reasonable_date]
    if not invalid_earlier_dates.empty:
        print(f"\nRows with {col} before {earliest_reasonable_date.date()}:")
        display(invalid_earlier_dates)

    # Check for dates after the latest reasonable date
    invalid_later_dates = appointment[appointment[col] > latest_reasonable_date]
    if not invalid_later_dates.empty:
        print(f"\nRows with {col} after {latest_reasonable_date.date()}:")
        display(invalid_later_dates)

    # Check for NaT values (already done by .info(), but can explicitly check if needed)
    nat_count = appointment[col].isnull().sum()
    if nat_count > 0:
        print(f"\nNumber of NaT values in {col}: {nat_count}")

# Check if ScheduledEnd is before ScheduledStart
invalid_scheduled_times = appointment[appointment['ScheduledEnd'] < appointment['ScheduledStart']]
if not invalid_scheduled_times.empty:
    print("\nRows where ScheduledEnd is before ScheduledStart:")
    display(invalid_scheduled_times)
else:
    print("\nAll ScheduledEnd dates are after ScheduledStart dates.")

# Check if ActualStart is before Arrival
invalid_arrival_times = appointment[appointment['ActualStart'] < appointment['Arrival']]
if not invalid_arrival_times.empty:
    print("\nRows where ActualStart is before Arrival:")
    display(invalid_arrival_times)
else:
    print("\nAll ActualStart dates are after Arrival dates.")


# Check if ActualEnd is before ActualStart (invalid)
invalid_actual_times = appointment[appointment['ActualEnd'] < appointment['ActualStart']]
if not invalid_actual_times.empty:
    print("\nRows where ActualEnd is before ActualStart:")
    display(invalid_actual_times)
else:
    print("\nAll ActualEnd dates are after ActualStart dates.")

Earliest Scheduled Start Date: 2020-01-01 09:00:00
Latest Scheduled End Date: 2024-12-31 17:00:00

Number of NaT values in Arrival: 2048

Number of NaT values in ActualStart: 2048

Number of NaT values in ActualEnd: 2048

All ScheduledEnd dates are after ScheduledStart dates.

All ActualStart dates are after Arrival dates.

All ActualEnd dates are after ActualStart dates.


#### Weight and Temperature
---

In [25]:
# Check data types for Weight(kg) and Temperature(celcius)
print("Data types:")
print(appointment[['Weight(kg)', 'Temperature(celcius)']].dtypes)

# Check for non-numeric values if the dtype is 'object' (although it's currently float64)
# If the dtype was object, we would try converting to numeric and identify errors:
# non_numeric_weight = pd.to_numeric(appointment['Weight(kg)'], errors='coerce').isnull()
# non_numeric_temp = pd.to_numeric(appointment['Temperature(celcius)'], errors='coerce').isnull()
# print("\nRows with non-numeric Weight(kg):")
# display(appointment[non_numeric_weight])
# print("\nRows with non-numeric Temperature(celcius):")
# display(appointment[non_numeric_temp])

# Check for viable weight range (e.g., > 0 and within a reasonable upper limit)
# Define a reasonable range (these are examples and might need adjustment based on context)
min_weight = 0
max_weight = 500 # Assuming weight is in kg, 500 kg is a generous upper limit

invalid_weight = appointment[(appointment['Weight(kg)'] <= min_weight) | (appointment['Weight(kg)'] > max_weight)]
if not invalid_weight.empty:
    print(f"\nRows with Weight(kg) outside the range ({min_weight}kg - {max_weight}kg):")
    display(invalid_weight)
else:
    print(f"\nAll Weight(kg) values are within the viable range ({min_weight}kg - {max_weight}kg).")

# Check for viable temperature range (e.g., typically human body temperature range)
# Define a reasonable range in Celsius
min_temp = 30 # Assuming a very low body temperature
max_temp = 45 # Assuming a very high fever

invalid_temp = appointment[(appointment['Temperature(celcius)'] < min_temp) | (appointment['Temperature(celcius)'] > max_temp)]
if not invalid_temp.empty:
    print(f"\nRows with Temperature(celcius) outside the range ({min_temp}°C - {max_temp}°C):")
    display(invalid_temp)
else:
    print(f"\nAll Temperature(celcius) values are within the viable range ({min_temp}°C - {max_temp}°C).")

# Check for nulls in these columns (already done by .info(), but good to reiterate)
print("\nNull counts for Weight(kg) and Temperature(celcius):")
print(appointment[['Weight(kg)', 'Temperature(celcius)']].isnull().sum())

Data types:
Weight(kg)              float64
Temperature(celcius)    float64
dtype: object

All Weight(kg) values are within the viable range (0kg - 500kg).

All Temperature(celcius) values are within the viable range (30°C - 45°C).

Null counts for Weight(kg) and Temperature(celcius):
Weight(kg)              2048
Temperature(celcius)    2048
dtype: int64


#### Comments
---

In [26]:
# Check data type of the 'Comment' column
print("Data type of 'Comment' column:")
display(appointment['Comment'].dtype)

# Check for null values in 'Comment'
null_comments_count = appointment['Comment'].isnull().sum()
print(f"\nNumber of null comments: {null_comments_count}")

# Check for non-string data types in 'Comment' (if the dtype is object)
if appointment['Comment'].dtype == 'object':
    non_string_comments = appointment[appointment['Comment'].apply(lambda x: not isinstance(x, str) and pd.notna(x))]
    if not non_string_comments.empty:
        print("\nRows with non-string comments:")
        display(non_string_comments)
    else:
        print("\nAll non-null comments are strings.")

Data type of 'Comment' column:


dtype('O')


Number of null comments: 2048

All non-null comments are strings.


# Patient Data Analysis and Cleaning
---

In [27]:
patient

,PatientID,Name,Sex,Contact,email,Address,DOB
0,P0001,Kelsey Moore,female,39472495,kelsey.moore@gmail.com,"34892 Christopher Crossing, South Brooke, NH 1...",1997-05-05
1,P0002,William Perry,male,15351117,william.perry@hotmail.com,"94501 Bell Islands Suite 485, New Jessica, NJ ...",2007-06-06
2,P0003,Nicholas Jones,male,56648257,nicholas.jones@yahoo.com,"8841 Lewis Parkways Apt. 040, South Kristen, I...",1975-12-04
3,P0004,Theresa Smith,female,45807263,theresa.smith@outlook.com,"271 Jeffrey Freeway, Willisbury, ME 39487",1997-09-14
4,P0005,Tara Kennedy,female,84234563,tara.kennedy@protonmail.com,"64128 Rick Coves Apt. 284, Lake James, NJ 51964",2011-02-22
...,...,...,...,...,...,...,...
995,P0996,Tina Murphy,female,81426708,tina.murphy@hotmail.com,"3606 Davidson Plaza Apt. 750, Lake Devinmouth,...",2019-02-17
996,P0997,Raymond Reed,male,95263259,raymond.reed@gmail.com,"5765 Lee Lodge, Cynthiahaven, AZ 26552",2006-11-25
997,P0998,Jose Torres,male,15324888,jose.torres@gmail.com,"55770 Jill Islands, Mccoyberg, SD 44091",2018-05-20
998,P0999,Marvin Perry,male,16959015,marvin.perry@outlook.com,"6824 Eric Village Apt. 042, Austinville, MN 33803",2004-07-03


In [28]:

print("--- Checking for Duplicate Values in Each Column ---")

# Iterate over each column name
for column in patient.columns:
    # The .duplicated() method finds duplicate values.
    # .sum() counts how many there are.
    num_duplicates_in_col = patient[column].duplicated().sum()

    if num_duplicates_in_col > 0:
        print(f"Column '{column}' has {num_duplicates_in_col} duplicate value(s).")
    else:
        print(f"Column '{column}' has no duplicate values.")

--- Checking for Duplicate Values in Each Column ---
Column 'PatientID' has no duplicate values.
Column 'Name' has 10 duplicate value(s).
Column 'Sex' has 998 duplicate value(s).
Column 'Contact' has no duplicate values.
Column 'email' has no duplicate values.
Column 'Address' has no duplicate values.
Column 'DOB' has 29 duplicate value(s).


The result makes sense.

In [29]:
# Check unique values in the 'Sex' column of the patient DataFrame
unique_sex_values = patient['Sex'].unique()

print("Unique values in the 'Sex' column of the patient DataFrame:")
print(unique_sex_values)

Unique values in the 'Sex' column of the patient DataFrame:
['female' 'male']


In [30]:
import re

# Check for hyphens or dashes in the 'Contact' column
contact_contains_punctuation = patient['Contact'].astype(str).str.contains(r'[-–—]', regex=True)

if contact_contains_punctuation.any():
    print("\nRows in 'Contact' column containing hyphens or dashes:")
    # Display rows where punctuation is found
    display(patient[contact_contains_punctuation])
else:
    print("\n'Contact' column does not contain hyphens or dashes.")


'Contact' column does not contain hyphens or dashes.


In [31]:
# Check if the 'email' column contains '@'
email_contains_at = patient['email'].str.contains('@', na=False)

if email_contains_at.all():
    print("All email addresses in the 'email' column contain '@'.")
else:
    print("Some email addresses in the 'email' column do not contain '@'.")
    # Optionally, display rows with invalid email formats
    # display(patient[~email_contains_at])

All email addresses in the 'email' column contain '@'.


In [32]:
patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   PatientID  1000 non-null   object        
 1   Name       1000 non-null   object        
 2   Sex        1000 non-null   object        
 3   Contact    1000 non-null   int64         
 4   email      1000 non-null   object        
 5   Address    1000 non-null   object        
 6   DOB        1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 54.8+ KB


There are no null values.

# Staff Data Analysis and Cleaning
---

In [33]:
staff

,DoctorID,Name,Contact,Email,DepartmentID,Department
0,E0001,Grace Chen,78901234,grace.chen@abchospital.com,D0005,Pediatrics
1,E0002,Henry Adams,75678901,henry.adams@abchospital.com,D0002,Intensive Care Unit
2,E0003,Maya Patel,44567890,maya.patel@abchospital.com,D0008,Surgery
3,E0004,Sean Underwood,84345678,sean.underwood@abchospital.com,D0001,Emergency
4,E0005,Amy Bryant,72123456,amy.bryant@abchospital.com,D0009,Neurology
...,...,...,...,...,...,...
95,E0096,Nina Turner,58273769,nina.turner@abchospital.com,D0004,Cardiology
96,E0097,Eric Hughes,12358576,eric.hughes@abchospital.com,D0009,Neurology
97,E0098,Lily Adams,29576793,lily.adams@abchospital.com,D0005,Pediatrics
98,E0099,Christopher Reed,31956390,christopher.reed@abchospital.com,D0010,Oncology


I will check for any duplicates based on each column.

In [34]:

print("--- Checking for Duplicate Values in Each Column ---")

# Iterate over each column name
for column in staff.columns:
    # The .duplicated() method finds duplicate values.
    # .sum() counts how many there are.
    num_duplicates_in_col = staff[column].duplicated().sum()

    if num_duplicates_in_col > 0:
        print(f"Column '{column}' has {num_duplicates_in_col} duplicate value(s).")
    else:
        print(f"Column '{column}' has no duplicate values.")

--- Checking for Duplicate Values in Each Column ---
Column 'DoctorID' has no duplicate values.
Column 'Name' has no duplicate values.
Column 'Contact' has no duplicate values.
Column 'Email' has no duplicate values.
Column 'DepartmentID' has 90 duplicate value(s).
Column 'Department' has 89 duplicate value(s).


We can say this dataset is free from duplicates, it makes sense that department are duplicated across doctors. I will examine more on these two columns having duplicates.

In [35]:


# Iterate over each column name
for column in staff.columns:
    # Check if any duplicates exist in the column
    if staff[column].duplicated().any():
        # Get the counts of each value in the column
        value_counts = staff[column].value_counts()

        # Filter to find only the values that appear more than once
        duplicated_values = value_counts[value_counts > 1]

        print(f"\nColumn '{column}' has duplicates.")
        print("   The following values are repeated:")
        # Print the series of duplicated values and their counts, indented for clarity
        print(duplicated_values.to_string(header=False).replace('\n', '\n   '))

    else:
        print(f"\nColumn '{column}' has no duplicate values.")




Column 'DoctorID' has no duplicate values.

Column 'Name' has no duplicate values.

Column 'Contact' has no duplicate values.

Column 'Email' has no duplicate values.

Column 'DepartmentID' has duplicates.
   The following values are repeated:
D0004    15
   D0009    12
   D0007    12
   D0005    11
   D0010    10
   D0001    10
   D0006     9
   D0002     8
   D0003     7
   D0008     6

Column 'Department' has duplicates.
   The following values are repeated:
Cardiology                   15
   Neurology                    12
   Obstetrics and Gynecology    12
   Pediatrics                   11
   Oncology                     10
   Emergency                     9
   Orthopedics                   9
   Intensive Care Unit           8
   Radiology                     7
   Surgery                       6


In fact, the counts must match across these two columns.

In [36]:
staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DoctorID      100 non-null    object
 1   Name          100 non-null    object
 2   Contact       100 non-null    int64 
 3   Email         100 non-null    object
 4   DepartmentID  100 non-null    object
 5   Department    100 non-null    object
dtypes: int64(1), object(5)
memory usage: 4.8+ KB


### Checking matches against facility dataset

In [37]:
# Check data types before cleaning
print("=== DATA TYPES BEFORE CLEANING ===")
print("Facility dataset:")
print(f"  Department: {facility['Department'].dtype}")

print("\nStaff dataset:")
print(f"  DepartmentID: {staff['DepartmentID'].dtype}")
print(f"  Department: {staff['Department'].dtype}")

print("\n" + "="*50)

# Clean whitespaces
print("=== CLEANING WHITESPACES ===")

# Facility dataset
facility['Department'] = facility['Department'].str.strip()

# Staff dataset
staff['DepartmentID'] = staff['DepartmentID'].str.strip()
staff['Department'] = staff['Department'].str.strip()

print("Whitespace cleaning completed!")

print("\n" + "="*50)

# Check data types after cleaning
print("=== DATA TYPES AFTER CLEANING ===")
print("Facility dataset:")
print(f"  Department: {facility['Department'].dtype}")

print("\nStaff dataset:")
print(f"  DepartmentID: {staff['DepartmentID'].dtype}")
print(f"  Department: {staff['Department'].dtype}")

=== DATA TYPES BEFORE CLEANING ===
Facility dataset:
  Department: object

Staff dataset:
  DepartmentID: object
  Department: object

=== CLEANING WHITESPACES ===
Whitespace cleaning completed!

=== DATA TYPES AFTER CLEANING ===
Facility dataset:
  Department: object

Staff dataset:
  DepartmentID: object
  Department: object


In [38]:
# --- Department Consistency Check ---
print("\n=== Department Consistency Check (Facility vs Staff) ===")

# Get unique department names from both dataframes
unique_departments_facility = facility['Department'].unique()
unique_departments_staff = staff['Department'].unique()

# Check if the sets of unique department names are the same
departments_match = set(unique_departments_facility) == set(unique_departments_staff)

print(f"Do the unique department names in facility and staff match? {departments_match}")

if not departments_match:
    # Find departments only in facility
    departments_only_in_facility = set(unique_departments_facility) - set(unique_departments_staff)
    if departments_only_in_facility:
        print("\nDepartments found only in facility:")
        print(departments_only_in_facility)

    # Find departments only in staff
    departments_only_in_staff = set(unique_departments_staff) - set(unique_departments_facility)
    if departments_only_in_staff:
        print("\nDepartments found only in staff:")
        print(departments_only_in_staff)


=== Department Consistency Check (Facility vs Staff) ===
Do the unique department names in facility and staff match? False

Departments found only in staff:
{'Emmergency'}


In [39]:
print("Unique Department entries in Facility dataset:")
display(facility['Department'].unique())

print("\nUnique Department entries in Staff dataset:")
display(staff['Department'].unique())

Unique Department entries in Facility dataset:


array(['Emergency', 'Cardiology', 'Pediatrics', 'Orthopedics',
       'Obstetrics and Gynecology', 'Surgery', 'Neurology', 'Oncology',
       'Radiology', 'Intensive Care Unit'], dtype=object)


Unique Department entries in Staff dataset:


array(['Pediatrics', 'Intensive Care Unit', 'Surgery', 'Emergency',
       'Neurology', 'Cardiology', 'Obstetrics and Gynecology', 'Oncology',
       'Orthopedics', 'Radiology', 'Emmergency'], dtype=object)

In [40]:
# Filter staff dataframe for 'Emergency' and 'Emmergency' departments
emergency_departments = staff[staff['Department'].isin(['Emergency', 'Emmergency'])]

# Display the Department and DepartmentID for these rows
print("Department and DepartmentID for 'Emergency' and 'Emmergency' in Staff dataset:")
display(emergency_departments[['Department', 'DepartmentID']])

Department and DepartmentID for 'Emergency' and 'Emmergency' in Staff dataset:


,Department,DepartmentID
3,Emergency,D0001
10,Emergency,D0001
27,Emergency,D0001
32,Emergency,D0001
40,Emergency,D0001
52,Emergency,D0001
63,Emergency,D0001
73,Emergency,D0001
83,Emmergency,D0001
91,Emergency,D0001


As we observe, the reason behind mismatch is because of spelling mistakes,

In [41]:
# Replace 'Emmergency' with 'Emergency' in the 'Department' column of the staff DataFrame
staff['Department'] = staff['Department'].replace('Emmergency', 'Emergency')

# Count unique values in the 'Department' column to verify the change
print("Unique values and their counts in the 'Department' column after correction:")
display(staff['Department'].value_counts())

Unique values and their counts in the 'Department' column after correction:


,count
Department,
Cardiology,15
Neurology,12
Obstetrics and Gynecology,12
Pediatrics,11
Oncology,10
Emergency,10
Orthopedics,9
Intensive Care Unit,8
Radiology,7


In [42]:
# Group by Department and check if the number of unique DepartmentIDs is 1 for each group
department_id_consistency = staff.groupby('Department')['DepartmentID'].nunique()

print("Number of unique DepartmentIDs per Department:")
display(department_id_consistency)

# Check if all departments have only one unique DepartmentID
all_consistent = (department_id_consistency == 1).all()

if all_consistent:
    print("\nDepartmentIDs are consistent for each Department name.")
else:
    print("\nInconsistency found: Some Departments have multiple DepartmentIDs.")
    # Optionally, display the departments with inconsistencies
    inconsistent_departments = department_id_consistency[department_id_consistency > 1]
    print("\nDepartments with inconsistent DepartmentIDs:")
    display(inconsistent_departments)

Number of unique DepartmentIDs per Department:


,DepartmentID
Department,
Cardiology,1
Emergency,1
Intensive Care Unit,1
Neurology,1
Obstetrics and Gynecology,1
Oncology,1
Orthopedics,1
Pediatrics,1
Radiology,1



DepartmentIDs are consistent for each Department name.


In [43]:
staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DoctorID      100 non-null    object
 1   Name          100 non-null    object
 2   Contact       100 non-null    int64 
 3   Email         100 non-null    object
 4   DepartmentID  100 non-null    object
 5   Department    100 non-null    object
dtypes: int64(1), object(5)
memory usage: 4.8+ KB


# Importing cleaned csv


checking before merging staff and facility for room dataset

In [44]:
# Get unique values from both columns
facility_departments = set(facility['Department'].unique())
staff_departments = set(staff['Department'].unique())

# Check if they're the same
are_same = facility_departments == staff_departments
print(f"Are the unique values the same? {are_same}")

Are the unique values the same? True


In [45]:
# Create department mapping from staff dataset
dept_mapping = staff[['Department', 'DepartmentID']].drop_duplicates()
dept_mapping = dict(zip(dept_mapping['Department'], dept_mapping['DepartmentID']))

# Add DepartmentID to facility dataset
facility['DepartmentID'] = facility['Department'].map(dept_mapping)

# Create room_cleaned dataset
room_cleaned = facility[['BuildingID', 'RoomID', 'Purpose', 'DepartmentID']].copy()


In [46]:
# Rename columns to match SQL table structure
appointment_cleaned = appointment.rename(columns={
   'ScheduledEnd': 'ScheduledEND',
   'Arrival': 'ArrivalTime',
   'ActualStart': 'StartTime',
   'ActualEnd': 'FinishTime',
   'Weight(kg)': 'Weight',
   'Temperature(celcius)': 'Temperature',
   'Comment': 'DoctorComment'
}).copy()

# Select columns in the correct order for the SQL table
appointment_cleaned = appointment_cleaned[['PatientID', 'DoctorID', 'BuildingID', 'RoomID', 'ScheduledStart', 'ScheduledEND', 'ArrivalTime', 'StartTime', 'FinishTime', 'Weight', 'Temperature', 'DoctorComment']]

print("Appointment dataset cleaned and ready")

Appointment dataset cleaned and ready


In [47]:
# Patient Table
patient.rename(columns={
    'Name': 'PatientName',
    'email': 'Email'
}, inplace=True)

# Convert Contact to string to match VARCHAR(10) in SQL
patient['Contact'] = patient['Contact'].astype(str)

# Doctor Table
# Rename Name to DoctorName
doctor = staff.copy()
doctor.rename(columns={'Name': 'DoctorName'}, inplace=True)

# Convert Contact to string to match VARCHAR(10)
doctor['Contact'] = doctor['Contact'].astype(str)

# Drop the extra Department column since it's not in the SQL table
if 'Department' in doctor.columns:
  doctor.drop(columns=['Department'], inplace=True)

if 'DepartmentName' in doctor.columns:
  doctor.drop(columns=['DepartmentName'], inplace=True)


# Building Table
# Assuming your DataFrame is named `building`

# Rename typo column 'BuldingID' to 'BuildingID'
facility.rename(columns={'BuldingID': 'BuildingID'}, inplace=True)

# Keep only required columns for SQL table
building = facility[['BuildingID', 'BuildingName']]
building = building.drop_duplicates()

# Department Table
# Rename 'Department' column to 'DepartmentName'
staff.rename(columns={'Department': 'DepartmentName'}, inplace=True)

# Keep only DepartmentID and DepartmentName
department = staff[['DepartmentID', 'DepartmentName']]
department = department.drop_duplicates()

Downloading files all dataset is cleaned

In [48]:
# Define path to save the new CSV (Downloads folder)
# download_path = os.path.expanduser('~/Downloads/patient.csv')

# file path
patient_file = "cleaned_patient.csv"
doctor_file = "cleaned_doctor.csv"
building_file = "cleaned_building.csv"
department_file = "cleaned_department.csv"
room_file = "cleaned_room.csv"
appointment_file = "cleaned_appointment.csv"

# Save the updated DataFrame to CSV without the index column
patient.to_csv(patient_file, index=False)
doctor.to_csv(doctor_file, index=False)
building.to_csv(building_file, index=False)
department.to_csv(department_file, index=False)
room_cleaned.to_csv(room_file, index=False)
appointment_cleaned.to_csv(appointment_file, index=False)

print(f"Modified patient.csv saved to: \n-{patient_file} \n-{doctor_file} \n-{building_file} \n-{department_file} \n-{room_file} \n-{appointment_file}")
try:
    from google.colab import files
    print("\nClick the links below to download the files (Google Colab):")
    # These might not work if the files are very large or in a complex directory structure
    # You might need to manually download from the file explorer in Colab
    files.download(patient_file)
    files.download(doctor_file)
    files.download(building_file)
    files.download(department_file)
    files.download(room_file)
    files.download(appointment_file)
except ImportError:
    print("\nRunning in a local environment (likely VS Code). Files saved to the current directory.")
    print("You can find the files in your file explorer.")

Modified patient.csv saved to: 
-cleaned_patient.csv 
-cleaned_doctor.csv 
-cleaned_building.csv 
-cleaned_department.csv 
-cleaned_room.csv 
-cleaned_appointment.csv

Click the links below to download the files (Google Colab):


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
# import os

# # Assuming your DataFrame is named `doctor`

# # Rename Name to DoctorName
# staff.rename(columns={'Name': 'DoctorName'}, inplace=True)

# # Convert Contact to string to match VARCHAR(10)
# staff['Contact'] = staff['Contact'].astype(str)

# # Drop the extra Department column since it's not in the SQL table
# staff.drop(columns=['Department'], inplace=True)

# # Define path to save to Downloads folder
# download_path = os.path.expanduser('~/Downloads/doctor.csv')

# # Save modified DataFrame to CSV
# staff.to_csv(download_path, index=False)

# print(f"Modified doctor.csv saved to {download_path}")

#--------------------------

# import os

# # Assuming your DataFrame is named `building`

# # Rename typo column 'BuldingID' to 'BuildingID'
# facility.rename(columns={'BuldingID': 'BuildingID'}, inplace=True)

# # Keep only required columns for SQL table
# building = facility[['BuildingID', 'BuildingName']]

# # Define path to save to Downloads folder
# download_path = os.path.expanduser('~/Downloads/building.csv')

# # Save to CSV
# building.to_csv(download_path, index=False)

# print(f"Modified building.csv saved to {download_path}")

#-------
# import os



# # Rename 'Department' column to 'DepartmentName'
# staff.rename(columns={'Department': 'DepartmentName'}, inplace=True)

# # Keep only DepartmentID and DepartmentName
# department = staff[['DepartmentID', 'DepartmentName']]

# # Define path to save to Downloads folder
# download_path = os.path.expanduser('~/Downloads/department.csv')

# # Save to CSV
# department.to_csv(download_path, index=False)

# print(f"Modified department.csv saved to {download_path}")
